In [ ]:
from math import log2
import essentia.standard
import numpy as np
from collections import Counter
import os
from time import time
from collections import OrderedDict
import cover_similarity_measures as sims

<h1>=====================================</h1>

In [ ]:
def to_cents(song):
    cents = []
    for i, f in enumerate(song):
        if song[i] > 0:
            cents.append(1200 * log2(song[i]/55))
        else:
            cents.append(0)
    return np.array(cents)

In [ ]:
def to_semitones(cents):
    semitones=[]
    for i, f in enumerate(cents):
        semitones.append(cents[i]//100)
    return np.array(semitones)

In [ ]:
def map_into_single_octave(semitones):
    mapped=[]
    min_n=1
    max_n=12
    min_d = np.min(semitones[np.nonzero(semitones)])
    max_d= max(semitones)
    for i, f in enumerate(semitones):
        if semitones[i] > 0:
            mapped.append(((semitones[i]-min_d)*(max_n-min_n))//(max_d-min_d)+min_n)
        else:
            mapped.append(0)
    return np.array(mapped)

In [ ]:
def get_histogram(pitch_class):
    limitator=0
    histogram=[]
    while limitator < len(pitch_class):
        frame=[]
        ini=limitator
        fin=ini+49
        if(fin>len(pitch_class)-1):
            fin=len(pitch_class)-1
        counter=Counter(pitch_class[ini:fin])
        for i in range(12):
            frame.append(counter[i+1])
        for x in range(len(frame)):
            if frame[x] == 0:
                continue
            num = frame[x]-min(frame)
            denom = max(frame)-min(frame)
            frame[x] = num/denom
        histogram.append(frame)
        limitator+=50
    return np.array(histogram, dtype=np.single)

In [ ]:
def compute_descriptor(melody):
    cents = to_cents(melody)
    semitones = to_semitones(cents)
    mapped = map_into_single_octave(semitones)
    chroma = get_histogram(mapped)
    return chroma

In [ ]:
def compute_similarity(_chroma, _chroma_db):
    _crp = essentia.standard.ChromaCrossSimilarity()
    _crp_matrix = _crp(_chroma, _chroma_db)
    _score_matrix, _distance = essentia.standard.CoverSongSimilarity()(_crp_matrix)
    return _distance

In [ ]:
def compute_similarity2(_chroma, _chroma_db):
    sim_matrix = sims.cross_recurrent_plot(_chroma, _chroma_db)
    qmax, cost_matrix = sims.qmax_measure(sim_matrix)
    return qmax

<h1>=====================================</h1>

In [ ]:
melodies_path = 'dataset/chromas_sbbc/vocals_spleeter/'
#melodies_path = 'dataset/chromas_sbbc/vocals_demucs/'
#melodies_path = 'dataset2/chromas_sbbc/vocals_spleeter/'
#melodies_path = 'dataset2/chromas_sbbc/vocals_demucs/'
hummings_path = 'dataset/hummings/'
#hummings_path = 'dataset2/hummings/'
melodies = os.listdir(melodies_path)
hummings = os.listdir(hummings_path)

In [ ]:
numerador_MRR = 0
numerador_TOP20 = 0
numerador_TOP15 = 0
numerador_TOP10 = 0
numerador_TOP5 = 0
numerador_TOP1 = 0
denominador = 0
tiempo = 0
for hum in hummings:
    denominador += 1
    results = {}
    print("........" + hum + "........")
    print("----------")
    start_time=time()
    loader = essentia.standard.EqloudLoader(filename = hummings_path + hum, sampleRate=44100)
    audio = loader()
    pitch_extractor = essentia.standard.PitchMelodia(frameSize=2048, hopSize=180)
    pitch_values, _ = pitch_extractor(audio)
    chroma = compute_descriptor(pitch_values)
    for melodie in melodies:
        chroma_db = np.loadtxt(melodies_path + melodie)
        results[melodie] = compute_similarity(chroma_db, chroma)
    ordered_results = OrderedDict({k: v for k, v in sorted(results.items(), key=lambda item: item[1])})
    tiempo += (time()-start_time)
    for i, rank in enumerate(ordered_results.items()):
        if hum.split('.')[0].split('+')[0] == rank[0].split('.')[0]:
            numerador_MRR += (1/(i+1))
            if (i+1) <= 1:
                numerador_TOP1 += 1
            if (i+1) <= 5:
                numerador_TOP5 += 1
            if (i+1) <= 10:
                numerador_TOP10 += 1
            if (i+1) <= 15:
                numerador_TOP15 += 1
            if (i+1) <= 20:
                numerador_TOP20 += 1
        print(i+1, rank[0], rank[1])
        print("----------")
    print("================================")
mrr = numerador_MRR / denominador
top1 = numerador_TOP1 / denominador * 100
top5 = numerador_TOP5 / denominador * 100
top10 = numerador_TOP10 / denominador * 100
top15 = numerador_TOP15 / denominador * 100
top20 = numerador_TOP20 / denominador * 100
tiempo = tiempo / denominador
print('MRR: ', mrr)
print('Top-1: ', top1)
print('Top-5: ', top5)
print('Top-10: ', top10)
print('Top-15: ', top15)
print('Top-20: ', top20)
print('Tiempo: ', tiempo)